In [1]:
import pandas as pd
from datetime import datetime
import sqlite3

In [2]:
df = pd.read_csv("rt_trips_DB_2018.txt", sep=";")

In [3]:
df.head(10)

,DATASOURCE,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,BASIN,TENDERLOT,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,DB,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,DB,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,DB,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,DB,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,DB,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2526331,"
5,DB,07-FEB-18 00:00:00,6248240,77A,77A_28,2,41648,37200,42019.0,37538.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2966500,"
6,DB,07-FEB-18 00:00:00,6251760,39,39_21,2,34768,28920,35709.0,28929.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2422850,"
7,DB,07-FEB-18 00:00:00,6262909,16,16_20,1,43936,38880,43721.0,38955.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",3045261,"
8,DB,07-FEB-18 00:00:00,6251147,40D,40D_102,1,33558,30600,34540.0,31185.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2837123,"
9,DB,07-FEB-18 00:00:00,6249435,27B,27B_34,2,52682,49800,53281.0,49974.0,BasDef,NaN,NaN,NaN,28-FEB-18 12:05:11,",2329338,"


In [5]:
df.shape

(2182637, 16)

In [6]:
df.dtypes

DATASOURCE          object
DAYOFSERVICE        object
TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
BASIN               object
TENDERLOT          float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE          object
NOTE                object
dtype: object

In [7]:
print('Number of duplicate rows (excluding first occurrence) in the table is:', df.duplicated().sum())

Number of duplicate rows (excluding first occurrence) in the table is: 0


In [8]:
# Display count of unique values for each feature
df.nunique(axis=0)

DATASOURCE              1
DAYOFSERVICE          360
TRIPID             658964
LINEID                130
ROUTEID               588
DIRECTION               2
PLANNEDTIME_ARR     64461
PLANNEDTIME_DEP       791
ACTUALTIME_ARR      68122
ACTUALTIME_DEP      66771
BASIN                   1
TENDERLOT               0
SUPPRESSED              1
JUSTIFICATIONID      3526
LASTUPDATE            360
NOTE                46690
dtype: int64

In [8]:
# Print unique values for each feature with 0 or 1 unique values - can these features be dropped?
print("UNIQUE VALUES")
print("DATASOURCE:", pd.unique(df["DATASOURCE"].ravel()))
print("BASIN:", pd.unique(df["BASIN"].ravel()))
print("TENDERLOT:", pd.unique(df["TENDERLOT"].ravel()))
print("SUPPRESSED:", pd.unique(df["SUPPRESSED"].ravel()))

UNIQUE VALUES
DATASOURCE: ['DB']
BASIN: ['BasDef']
TENDERLOT: [nan]
SUPPRESSED: [nan  0.]


In [9]:
# Drop DATASOURCE, BASIN, TENDERLOT - unsure of use of SUPPRESSED so keeping for now
df.drop(["DATASOURCE", "BASIN", "TENDERLOT"], axis = 1, inplace=True)

In [10]:
df.head()

,DAYOFSERVICE,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE
0,07-FEB-18 00:00:00,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,NaN,28-FEB-18 12:05:11,",2967409,"
1,07-FEB-18 00:00:00,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,NaN,28-FEB-18 12:05:11,",2580260,"
2,07-FEB-18 00:00:00,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,NaN,28-FEB-18 12:05:11,",2448968,"
3,07-FEB-18 00:00:00,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,NaN,28-FEB-18 12:05:11,",3094242,"
4,07-FEB-18 00:00:00,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,NaN,28-FEB-18 12:05:11,",2526331,"


In [11]:
# Define function to convert datetime in format seen in data to timestamp
def to_timestamp(date):
    stamp = int(datetime.timestamp(datetime.strptime(date, '%d-%b-%y %H:%M:%S')))
    return stamp

In [12]:
# Create new dt feature containing datetimes from DAYOFSERVICE converted to timestamp - for consistency across data
# Convert values in LASTUPDATE to timestamp
df["dt"] = df["DAYOFSERVICE"].apply(to_timestamp)
df["LASTUPDATE"] = df["LASTUPDATE"].apply(to_timestamp)

In [13]:
# Drop DAYOFSERVICE
df.drop(["DAYOFSERVICE"], axis = 1, inplace=True)

In [14]:
df.head()

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE,dt
0,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,NaN,1519819511,",2967409,",1517961600
1,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,NaN,1519819511,",2580260,",1517961600
2,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,NaN,1519819511,",2448968,",1517961600
3,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,NaN,1519819511,",3094242,",1517961600
4,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,NaN,1519819511,",2526331,",1517961600


In [15]:
# Convert values in NOTE to type string
df["NOTE"] = df["NOTE"].astype("string") 

In [16]:
df.dtypes

TRIPID               int64
LINEID              object
ROUTEID             object
DIRECTION            int64
PLANNEDTIME_ARR      int64
PLANNEDTIME_DEP      int64
ACTUALTIME_ARR     float64
ACTUALTIME_DEP     float64
SUPPRESSED         float64
JUSTIFICATIONID    float64
LASTUPDATE           int64
NOTE                string
dt                   int64
dtype: object

In [17]:
# Define function to strip commas from values in NOTE
def comma_strip(string):
    if isinstance(string, str):
        return string.strip(",")
    else:
        pass

In [18]:
# Apply function comma_strip to each value in NOTE
df["NOTE"] = df["NOTE"].apply(comma_strip)

In [19]:
df.head()

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE,dt
0,6253783,68,68_80,1,87245,84600,87524.0,84600.0,NaN,NaN,1519819511,2967409,1517961600
1,6262138,25B,25B_271,2,30517,26460,32752.0,NaN,NaN,NaN,1519819511,2580260,1517961600
2,6254942,45A,45A_70,2,35512,32100,36329.0,32082.0,NaN,NaN,1519819511,2448968,1517961600
3,6259460,25A,25A_273,1,57261,54420,58463.0,54443.0,NaN,NaN,1519819511,3094242,1517961600
4,6253175,14,14_15,1,85383,81600,84682.0,81608.0,NaN,NaN,1519819511,2526331,1517961600


In [20]:
df.to_csv("rt_trips_DB_2018_cleaned.csv", index=False)

<h3>Sample for Download</h3>

In [21]:
# Define function to convert datetime in format seen in data to timestamp
def to_timestamp2(x):
    out = int(datetime.timestamp(datetime.strptime(x, '%d-%b-%Y %H:%M')))
    return out

In [71]:
start_date = to_timestamp2("05-mar-2018 00:00")
end_date = to_timestamp2("11-mar-2018 23:00") 
mar_df = df[df["dt"] >= start_date]
mar_df.head(5)

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE,dt
169,6397950,27X,27X_43,1,66778,63000,66753.0,62966.0,NaN,NaN,1522065343,2443660,1521158400
170,6397972,18,18_3,1,71941,67800,NaN,67622.0,NaN,NaN,1522065343,1751223,1521158400
171,6398012,145,145_102,1,59653,54600,60906.0,54684.0,NaN,NaN,1522065343,2416530,1521158400
172,6594815,122,122_16,2,28100,24300,27774.0,24355.0,NaN,NaN,1522832356,2760723,1522105200
173,6388510,122,122_16,2,75168,72000,75138.0,72053.0,NaN,NaN,1521471779,1732206,1520726400


In [72]:
mar_df.shape

(1805753, 13)

In [73]:
mar_df = mar_df[(mar_df["dt"]) <= end_date]
print("Number of rows in subset: ", mar_df.shape[0])

Number of rows in subset:  43935


In [74]:
mar_df.head()

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE,dt
173,6388510,122,122_16,2,75168,72000,75138.0,72053.0,NaN,NaN,1521471779,1732206,1520726400
174,6391964,25A,25A_270,2,33266,30720,33414.0,30728.0,NaN,NaN,1521471779,2285916,1520726400
175,6391924,25A,25A_270,2,35066,32520,35238.0,32529.0,NaN,NaN,1521471779,2289998,1520726400
176,6401718,25A,25A_270,2,36866,34320,36996.0,34327.0,NaN,NaN,1521471779,2284142,1520726400
177,6401563,122,122_17,2,86104,84600,85983.0,84623.0,NaN,NaN,1521471779,1749068,1520726400


In [75]:
mar_df.sort_values(by="dt", inplace=True)

In [76]:
mar_df.tail()

,TRIPID,LINEID,ROUTEID,DIRECTION,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,JUSTIFICATIONID,LASTUPDATE,NOTE,dt
20296,6396886,40D,40D_103,2,46551,44100,46737.0,44087.0,NaN,NaN,1521471779,1741197,1520726400
20295,6396938,40D,40D_103,2,49551,47100,50064.0,47142.0,NaN,NaN,1521471779,1753965,1520726400
20294,6401637,40D,40D_103,2,52551,50100,53352.0,50096.0,NaN,NaN,1521471779,1751115,1520726400
20292,6392309,43,43_84,1,51300,48600,51974.0,48900.0,NaN,NaN,1521471779,2324606,1520726400
173,6388510,122,122_16,2,75168,72000,75138.0,72053.0,NaN,NaN,1521471779,1732206,1520726400


In [77]:
mar_df.shape

(43935, 13)

In [78]:
mar_df.to_csv("5_11_march_trips.csv", index=False)